## Dependencies

In [1]:
# Install Modules
!pip install keras_tuner -q

# Load Dependencies
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import tensorflow as tf
import keras_tuner as kt
import warnings
import numpy as np
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import time

# Disable all warnings
warnings.filterwarnings("ignore")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 4.9 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
def print_loading_bar(current, total, time, bar_length=40):
    progress = current / total
    completed_length = int(bar_length * progress)
    remaining_length = bar_length - completed_length

    bar = '█' * completed_length + '-' * remaining_length

    print(f"\r[{bar}] Processing {current} of {total} COMPLETE {progress*100:.2f}% Time Lapsed {time:.2f} Seconds", end ="")

## Load Data

In [20]:
def load_join(select, loaded):
    selected_df = pd.read_csv("https://www.dropbox.com/s/" + select + "?dl=1", index_col="review_index")
    if loaded is not None:
        selected_df = pd.merge(loaded, selected_df, on='review_index')
    return selected_df

In [58]:
load_metadata = True
load_ratings = True
load_opinions = True
load_descriptions = True
load_world_happiness_report = False
word_counted_preprocessing_bypass = True

index_df = None
if load_metadata == True:
    load_index = "goqyfyeu2qvpsmo/metadata_df.csv"
    index_df = load_join(load_index, index_df)
if load_ratings == True:
    load_index = "9j2j86xwqrmrljx/ratings_df.csv"
    index_df = load_join(load_index, index_df)
if load_opinions == True:
    load_index = "q8v9f6rbb4z12df/opinion_df.csv"
    index_df = load_join(load_index, index_df)
if load_descriptions == True:
    load_index = "hnp8sebleh6dzgt/descriptions_df.csv"
    index_df = load_join(load_index, index_df)
    
if load_world_happiness_report == True:
    WHR_df = pd.read_csv("https://www.dropbox.com/s/jyr4e7fleevrb1s/WHR2023.csv?dl=1")

if word_counted_preprocessing_bypass == True:
    joint_df = pd.read_csv("https://www.dropbox.com/s/e64gm79gxu1tgip/joint_df.csv?dl=1")

In [22]:
user_input_df = index_df.drop(['firm', 'date_review', 'job_title', 'location', 'current',	'work_life_balance',	'culture_values',	'diversity_inclusion',
                               'career_opp',	'comp_benefits',	'senior_mgmt',	'recommend',	'ceo_approv',	'outlook'], axis=1).copy()

nn_working_df = index_df.drop(['date_review', 'job_title', 'location', 'headline', 'pros', 'cons'], axis=1).copy()

nn_op_working_df = index_df.drop(['date_review', 'job_title', 'location'], axis=1).copy()

## Cleaning Data Attempt - Half Hot-Ended Remove NaN

In [23]:
working_half_hotend_df = nn_working_df.copy()
columns_to_convert = ['overall_rating', 'work_life_balance', 'culture_values',
                      'diversity_inclusion', 'career_opp', 'comp_benefits',
                      'senior_mgmt']

working_half_hotend_df[columns_to_convert] = working_half_hotend_df[columns_to_convert].astype(float)

In [24]:
cutoff_value = 3000
firms_to_replace = working_half_hotend_df['firm'].value_counts()[working_half_hotend_df['firm'].value_counts() < cutoff_value].index.tolist()

# Replace in dataframe
for firm in firms_to_replace:
    working_half_hotend_df['firm'] = working_half_hotend_df['firm'].replace(firm,"Other")
    
# Check to make sure binning was successful
print(working_half_hotend_df['firm'].value_counts())
print("These ar how many samples that got 'firm' changed to Other: ", len(working_half_hotend_df[working_half_hotend_df['firm'] == "Other"]))
print("Number of firms removed: ", len(firms_to_replace))
print("Remaining number of firms: ", len(working_half_hotend_df['firm'].value_counts()))

Other                      178782
IBM                         60436
McDonald-s                  49450
Deloitte                    46995
EY                          34050
PwC                         33227
Oracle                      31941
Microsoft                   26675
J-P-Morgan                  25814
KPMG                        24815
Apple                       20797
Citi                        18726
Google                      15995
SAP                         14344
HSBC-Holdings               13893
Tesco                       12149
Marriott-International      10409
Barclays                     9710
Thomson-Reuters              9553
American-Express             9349
Morgan-Stanley               9093
Goldman-Sachs                8808
Vodafone                     8321
Salesforce                   8234
Pizza-Hut                    7592
BNY-Mellon                   6630
Deutsche-Bank                6388
Hilton                       6155
J-Sainsbury                  5925
GlaxoSmithKlin

In [25]:
# Cutoff value to make a list of values in the 'current' column to put into "other" catagory
cutoff_value = 10
current_to_replace = working_half_hotend_df['current'].value_counts()[working_half_hotend_df['current'].value_counts() < cutoff_value].index.tolist()

# Replace in dataframe
for review in current_to_replace:
    working_half_hotend_df['current'] = working_half_hotend_df['current'].replace(review,"Other")

# Check to make sure binning was successful
working_half_hotend_df['current'].value_counts()

Current Employee                        209599
Former Employee                         146133
Current Employee, more than 1 year       82749
Current Employee, more than 3 years      66471
Former Employee, more than 1 year        65687
Current Employee, less than 1 year       49603
Former Employee, more than 3 years       43614
Former Employee, less than 1 year        41874
Current Employee, more than 5 years      40155
Current Employee, more than 10 years     25029
Former Employee, more than 5 years       23017
Current Employee, more than 8 years      18506
Former Employee, more than 10 years      15411
Former Employee, more than 8 years       10686
Other                                       32
Name: current, dtype: int64

In [26]:
print("Columns before processing: ", len(working_half_hotend_df))
working_half_hotend_df.dropna(subset=columns_to_convert, inplace=True)
print("Column after dropping NaN values: ", len(working_half_hotend_df))
working_half_hotend_df.nunique()

Columns before processing:  838566
Column after dropping NaN values:  133863


firm                   59
current                15
overall_rating          5
work_life_balance       5
culture_values          5
diversity_inclusion     5
career_opp              5
comp_benefits           5
senior_mgmt             5
recommend               3
ceo_approv              4
outlook                 4
dtype: int64

In [27]:
X = pd.get_dummies(working_half_hotend_df.drop('overall_rating', axis = 1)).drop(['recommend_o', 'ceo_approv_o', 'outlook_o'], axis = 1)
y = pd.get_dummies(working_half_hotend_df['overall_rating'].astype(int))

X.columns

Index(['work_life_balance', 'culture_values', 'diversity_inclusion',
       'career_opp', 'comp_benefits', 'senior_mgmt', 'firm_ASDA',
       'firm_American-Express', 'firm_Aon', 'firm_Apple', 'firm_AstraZeneca',
       'firm_BNP-Paribas', 'firm_BNY-Mellon', 'firm_BT',
       'firm_Bain-and-Company', 'firm_Barclays', 'firm_Bloomberg-L-P',
       'firm_Booking-com', 'firm_Boots', 'firm_Boston-Consulting-Group',
       'firm_CBRE', 'firm_Capita', 'firm_Citi', 'firm_Deloitte',
       'firm_Deutsche-Bank', 'firm_EY', 'firm_GlaxoSmithKline',
       'firm_Goldman-Sachs', 'firm_Google', 'firm_Grant-Thornton',
       'firm_HSBC-Holdings', 'firm_Hays', 'firm_Hilton', 'firm_Hyatt',
       'firm_IBM', 'firm_J-P-Morgan', 'firm_J-Sainsbury', 'firm_KPMG',
       'firm_LinkedIn', 'firm_Lloyds-Banking-Group', 'firm_Marks-and-Spencer',
       'firm_Marriott-International', 'firm_McDonald-s',
       'firm_McKinsey-and-Company', 'firm_Mercer', 'firm_Microsoft',
       'firm_Morgan-Stanley', 'firm_NHS', '

In [28]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

X_train_scaled.shape

(100397, 88)

## Cleaning Data Attempt - Half Hot-Ended Mask NaN

In [6]:
working_mask_df = nn_working_df.copy()
columns_to_convert = ['overall_rating', 'work_life_balance', 'culture_values',
                      'diversity_inclusion', 'career_opp', 'comp_benefits',
                      'senior_mgmt']

working_mask_df[columns_to_convert] = working_mask_df[columns_to_convert].astype(float)

In [7]:
cutoff_value = 3000
firms_to_replace = working_mask_df['firm'].value_counts()[working_mask_df['firm'].value_counts() < cutoff_value].index.tolist()

# Replace in dataframe
for firm in firms_to_replace:
    working_mask_df['firm'] = working_mask_df['firm'].replace(firm,"Other")
    
# Check to make sure binning was successful
print(working_mask_df['firm'].value_counts())
print("These ar how many samples that got 'firm' changed to Other: ", len(working_mask_df[working_mask_df['firm'] == "Other"]))
print("Number of firms removed: ", len(firms_to_replace))
print("Remaining number of firms: ", len(working_mask_df['firm'].value_counts()))

Other                      178782
IBM                         60436
McDonald-s                  49450
Deloitte                    46995
EY                          34050
PwC                         33227
Oracle                      31941
Microsoft                   26675
J-P-Morgan                  25814
KPMG                        24815
Apple                       20797
Citi                        18726
Google                      15995
SAP                         14344
HSBC-Holdings               13893
Tesco                       12149
Marriott-International      10409
Barclays                     9710
Thomson-Reuters              9553
American-Express             9349
Morgan-Stanley               9093
Goldman-Sachs                8808
Vodafone                     8321
Salesforce                   8234
Pizza-Hut                    7592
BNY-Mellon                   6630
Deutsche-Bank                6388
Hilton                       6155
J-Sainsbury                  5925
GlaxoSmithKlin

In [8]:
# Cutoff value to make a list of values in the 'current' column to put into "other" catagory
cutoff_value = 10
current_to_replace = working_mask_df['current'].value_counts()[working_mask_df['current'].value_counts() < cutoff_value].index.tolist()

# Replace in dataframe
for review in current_to_replace:
    working_mask_df['current'] = working_mask_df['current'].replace(review,"Other")

# Check to make sure binning was successful
working_mask_df['current'].value_counts()

Current Employee                        209599
Former Employee                         146133
Current Employee, more than 1 year       82749
Current Employee, more than 3 years      66471
Former Employee, more than 1 year        65687
Current Employee, less than 1 year       49603
Former Employee, more than 3 years       43614
Former Employee, less than 1 year        41874
Current Employee, more than 5 years      40155
Current Employee, more than 10 years     25029
Former Employee, more than 5 years       23017
Current Employee, more than 8 years      18506
Former Employee, more than 10 years      15411
Former Employee, more than 8 years       10686
Other                                       32
Name: current, dtype: int64

In [9]:
X = pd.get_dummies(working_mask_df.drop('overall_rating', axis = 1))
X = X.fillna(X.mean())

y = pd.get_dummies(working_mask_df['overall_rating'].astype(int))

X.head()

,work_life_balance,culture_values,diversity_inclusion,career_opp,comp_benefits,senior_mgmt,firm_ASDA,firm_American-Express,firm_Aon,firm_Apple,...,recommend_v,recommend_x,ceo_approv_o,ceo_approv_r,ceo_approv_v,ceo_approv_x,outlook_o,outlook_r,outlook_v,outlook_x
review_index,,,,,,,,,,,,,,,,,,,,,
0,4.0,3.0,3.966612,2.0,3.0,3.0,0,0,0,0,...,0,1,1,0,0,0,0,1,0,0
1,3.0,1.0,3.966612,2.0,1.0,4.0,0,0,0,0,...,0,1,1,0,0,0,0,1,0,0
2,1.0,1.0,3.966612,1.0,1.0,1.0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,1
3,2.0,3.0,3.966612,2.0,2.0,3.0,0,0,0,0,...,0,1,1,0,0,0,0,1,0,0
4,2.0,1.0,3.966612,2.0,1.0,1.0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,1


In [10]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

X_train_scaled.shape

(628924, 91)

## Scaping Key Words from User Inputs

In [34]:
working_user_input_df = user_input_df.copy()
working_user_input_df = working_user_input_df[working_user_input_df.index.isin(working_half_hotend_df.index)] # working_mask_df is alternative
print("Rows in dataset prior ro removing rows with NaN values: ", len(user_input_df))
print("Rows in dataset after removing rows with NaN values: ", len(working_user_input_df))
working_user_input_df.head(1)

Rows in dataset prior ro removing rows with NaN values:  838566
Rows in dataset after removing rows with NaN values:  133863


,overall_rating,headline,pros,cons
review_index,,,,
50,2,The people both make and destroy this place,"Great people in some places, excellent Christm...","Poor pay, huge gap for pay between senior mana..."


In [35]:
def text_tokenizer(text):
    # Tokenize the text into individual words
    words = word_tokenize(text.lower())

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word.isalnum() and word not in stop_words]

    return filtered_words

# Text processor takes dataframe of only 1 column of strings, tokenizes it, and returns series of comma separated words from each string.

In [36]:
def word_counter(series_column, n):
  
  all_words = [word for sublist in series_column for word in sublist]
  word_counts = Counter(all_words)

  most_common_words = word_counts.most_common(n)

  return most_common_words

# Takes in tokenized series and returns the top "value of 'n'" words as a series.

In [37]:
def column_processor(rating, attribute, n):
  index_user_input_df = working_user_input_df[working_user_input_df['overall_rating'] == rating].copy().drop(['overall_rating'], axis = 1)
  isolated = index_user_input_df[attribute].rename(str(attribute)+ "_" + str(rating))
  isolated = isolated.reset_index(drop=True)
  isolated = isolated.dropna().astype(str)
  preprocessed_data = isolated.apply(text_tokenizer)
  word_count_list = word_counter(preprocessed_data, n)

  return word_count_list

# Takes requested rating and attribute, finds the column, and calls to the function that tokenizes text, calls to the function that counts words. Returns series.

In [39]:
parsed_df = pd.DataFrame()
start_time = time.time()
return_top_counter = 750

for i in range(1, 6):
    for attribute in working_user_input_df.drop(['overall_rating'], axis=1):
        index_series_to_df = pd.DataFrame(column_processor(i, attribute, return_top_counter), columns=[str(attribute) + '_' + str(i) + '_words', str(attribute) + '_' + str(i) + '_count'])
        index_series_to_df = index_series_to_df.rename_axis('word_index')
        if parsed_df.empty:
            parsed_df = index_series_to_df
        else:
            parsed_df = pd.merge(parsed_df, index_series_to_df, on='word_index')
    current_time = time.time()
    execution_time = current_time - start_time
    print_loading_bar(i, 5, execution_time)

[████████████████████████████████████████] Processing 5 of 5 COMPLETE 100.00% Time Lapsed 124.74 Seconds

In [40]:
parsed_words_df = parsed_df.drop(columns=[col for col in parsed_df.columns if 'count' in col])
duplicate_words_list = parsed_words_df.values.flatten().tolist()
duplicate_words_list = [word for word in duplicate_words_list if duplicate_words_list.count(word) > 1]
duplicate_words_list = list(set(duplicate_words_list))
print(len(duplicate_words_list))

1475


In [41]:
parsed_words_optimized_df = parsed_words_df.copy()
for column in parsed_words_optimized_df.columns:
    parsed_words_optimized_df[column] = parsed_words_optimized_df[column].apply(lambda x: '' if x in duplicate_words_list else x)

parsed_words_optimized_df = parsed_words_optimized_df.apply(lambda x: x.sort_values().values)
parsed_words_optimized_df = parsed_words_optimized_df.apply(lambda x: sorted(x, key=bool, reverse=True))
parsed_words_optimized_df = parsed_words_optimized_df[~(parsed_words_optimized_df == '').all(axis=1)]
parsed_words_optimized_df.head(10)

,headline_1_words,pros_1_words,cons_1_words,headline_2_words,pros_2_words,cons_2_words,headline_3_words,pros_3_words,cons_3_words,headline_4_words,pros_4_words,cons_4_words,headline_5_words,pros_5_words,cons_5_words
word_index,,,,,,,,,,,,,,,
0,abused,advantage,7,beginning,art,alone,adequate,availability,accountability,aon,active,adapt,2020,accommodating,applicable
1,administrative,air,abuse,british,assistance,asking,aight,camaraderie,external,ba,benifits,assigned,accounts,continue,cold
2,appalling,attractive,act,careful,cafe,cuts,builder,casual,inefficient,becoming,businesses,band,ae,cooperative,commute
3,atrocious,bearable,allow,carefully,deals,deliver,dependant,coverage,rise,commercial,competent,comparison,ai,developing,complain
4,azure,book,answer,carrier,direct,drama,downs,prestigious,stand,deutsche,cross,confusing,ambassador,encourage,complexity
5,backwards,buildings,behavior,cog,finish,heard,ehh,rates,,foundation,learnings,demand,apprentice,encourages,downside
6,bone,buy,bloomberg,debenhams,guaranteed,promised,losing,soft,,grind,organised,demands,army,ensure,downsides
7,brutal,country,board,draining,immediate,regardless,neutral,,,improvements,sectors,depend,awsome,equality,facing
8,burn,drink,bullied,execution,main,related,reporting,,,kickstart,transport,merit,bain,joined,fault


## Processing User Input strings into columns with word counts

In [45]:
nn_op_working_df[columns_to_convert] = nn_op_working_df[columns_to_convert].astype(float)

cutoff_value = 3000
firms_to_replace = nn_op_working_df['firm'].value_counts()[nn_op_working_df['firm'].value_counts() < cutoff_value].index.tolist()

for firm in firms_to_replace:
    nn_op_working_df['firm'] = nn_op_working_df['firm'].replace(firm,"Other")

cutoff_value2 = 10
current_to_replace = nn_op_working_df['current'].value_counts()[nn_op_working_df['current'].value_counts() < cutoff_value2].index.tolist()

for review in current_to_replace:
    nn_op_working_df['current'] = nn_op_working_df['current'].replace(review,"Other")

nn_op_working_df.dropna(subset=columns_to_convert, inplace=True) # Pound out to work with full dataset with NaN values.
nn_op_working_df = nn_op_working_df.reset_index(drop=True)
print(len(nn_op_working_df))

133863


In [46]:
start_time = time.time()
word_counter_df = pd.DataFrame() # Full DF to be merged

for index, row in nn_op_working_df.iterrows():
    index_df = pd.DataFrame() # Individual row
    if index % 1000 == 0 or index == len(nn_op_working_df):
        current_time = time.time()
        execution_time = current_time - start_time
        print_loading_bar(index, len(nn_op_working_df),execution_time)
    for column in ['headline', 'pros', 'cons']:
        attribute_index_df = pd.DataFrame() # column in Individual row
        working_index = text_tokenizer(str(row[column]))
        for i in range(1, 6):
            attribute_rating_counter = 0 # Value to be acdded
            checking_index = parsed_words_df[parsed_words_df[str(column) + '_' + str(i) + '_words'] != ''][str(column) + '_' + str(i) + '_words'].tolist()
            for word in working_index:
                if word in checking_index: # count instances of matching words
                    attribute_rating_counter = attribute_rating_counter + 1
            attribute_index_df[str(column) + '_' + str(i) + '_counter'] = [attribute_rating_counter] # creates a new column for each attribute and rating counter
        if index_df.empty:
            index_df = attribute_index_df
        else:
            index_df = pd.merge(index_df, attribute_index_df, left_index=True, right_index=True)
    word_counter_df = pd.concat([word_counter_df, index_df], ignore_index=True)

word_counter_df

[███████████████████████████████████████-] Processing 133000 of 133863 COMPLETE 99.36% Time Lapsed 4139.74 Seconds

,headline_1_counter,headline_2_counter,headline_3_counter,headline_4_counter,headline_5_counter,pros_1_counter,pros_2_counter,pros_3_counter,pros_4_counter,pros_5_counter,cons_1_counter,cons_2_counter,cons_3_counter,cons_4_counter,cons_5_counter
0,3,3,3,3,3,5,5,5,5,4,15,14,15,15,15
1,2,2,2,2,2,4,4,3,3,3,24,26,26,26,27
2,1,1,1,1,1,4,4,5,4,4,2,2,2,2,2
3,1,1,1,1,1,3,3,3,4,4,6,7,8,8,7
4,3,3,3,3,3,7,7,7,7,7,3,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133858,2,2,2,2,2,5,5,5,5,5,8,9,8,8,8
133859,2,2,3,3,3,4,5,7,7,8,3,4,4,4,4
133860,2,2,3,3,3,9,10,10,10,10,0,0,1,2,2
133861,1,1,2,2,2,3,4,4,4,4,8,8,8,8,6


In [57]:
joint_df = working_half_hotend_df.copy()
print(joint_df.shape)
joint_df.reset_index(drop=True, inplace=True)
joint_df = pd.merge(joint_df, word_counter_df, left_index=True, right_index=True)
print(joint_df.shape)
joint_df.to_csv('joint_df.csv', index=False)
joint_df.head(3)

(133863, 12)
(133863, 27)


,firm,current,overall_rating,work_life_balance,culture_values,diversity_inclusion,career_opp,comp_benefits,senior_mgmt,recommend,ceo_approv,outlook,headline_1_counter,headline_2_counter,headline_3_counter,headline_4_counter,headline_5_counter,pros_1_counter,pros_2_counter,pros_3_counter,pros_4_counter,pros_5_counter,cons_1_counter,cons_2_counter,cons_3_counter,cons_4_counter,cons_5_counter
0,Other,"Former Employee, more than 3 years",2.0,1.0,3.0,1.0,1.0,2.0,2.0,x,o,x,3,3,3,3,3,5,5,5,5,4,15,14,15,15,15
1,Other,Former Employee,1.0,3.0,1.0,2.0,1.0,1.0,1.0,x,o,o,2,2,2,2,2,4,4,3,3,3,24,26,26,26,27
2,Other,"Former Employee, less than 1 year",4.0,3.0,3.0,4.0,4.0,4.0,2.0,v,o,v,1,1,1,1,1,4,4,5,4,4,2,2,2,2,2


In [59]:
# bypass
# Do not do anything before this if joint_df is loaded

X = pd.get_dummies(joint_df.drop('overall_rating', axis = 1))#.drop(['recommend_o', 'ceo_approv_o', 'outlook_o'], axis = 1)
y = pd.get_dummies(joint_df['overall_rating'].astype(int))

pd.set_option('display.max_columns', None)  # Show all columns
X.head(10)

,work_life_balance,culture_values,diversity_inclusion,career_opp,comp_benefits,senior_mgmt,headline_1_counter,headline_2_counter,headline_3_counter,headline_4_counter,headline_5_counter,pros_1_counter,pros_2_counter,pros_3_counter,pros_4_counter,pros_5_counter,cons_1_counter,cons_2_counter,cons_3_counter,cons_4_counter,cons_5_counter,firm_ASDA,firm_American-Express,firm_Aon,firm_Apple,firm_AstraZeneca,firm_BNP-Paribas,firm_BNY-Mellon,firm_BT,firm_Bain-and-Company,firm_Barclays,firm_Bloomberg-L-P,firm_Booking-com,firm_Boots,firm_Boston-Consulting-Group,firm_CBRE,firm_Capita,firm_Citi,firm_Deloitte,firm_Deutsche-Bank,firm_EY,firm_GlaxoSmithKline,firm_Goldman-Sachs,firm_Google,firm_Grant-Thornton,firm_HSBC-Holdings,firm_Hays,firm_Hilton,firm_Hyatt,firm_IBM,firm_J-P-Morgan,firm_J-Sainsbury,firm_KPMG,firm_LinkedIn,firm_Lloyds-Banking-Group,firm_Marks-and-Spencer,firm_Marriott-International,firm_McDonald-s,firm_McKinsey-and-Company,firm_Mercer,firm_Microsoft,firm_Morgan-Stanley,firm_NHS,firm_Oracle,firm_Other,firm_Pearson,firm_Pizza-Hut,firm_Primark,firm_PwC,firm_SAP,firm_Salesforce,firm_Sodexo,firm_Tesco,firm_Thomson-Reuters,firm_Unilever,firm_University-of-Michigan,firm_Vodafone,firm_Waitrose,firm_Willis-Towers-Watson,firm_next,current_Current Employee,"current_Current Employee, less than 1 year","current_Current Employee, more than 1 year","current_Current Employee, more than 10 years","current_Current Employee, more than 3 years","current_Current Employee, more than 5 years","current_Current Employee, more than 8 years",current_Former Employee,"current_Former Employee, less than 1 year","current_Former Employee, more than 1 year","current_Former Employee, more than 10 years","current_Former Employee, more than 3 years","current_Former Employee, more than 5 years","current_Former Employee, more than 8 years",current_Other,recommend_o,recommend_v,recommend_x,ceo_approv_o,ceo_approv_r,ceo_approv_v,ceo_approv_x,outlook_o,outlook_r,outlook_v,outlook_x
0,1.0,3.0,1.0,1.0,2.0,2.0,3,3,3,3,3,5,5,5,5,4,15,14,15,15,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1
1,3.0,1.0,2.0,1.0,1.0,1.0,2,2,2,2,2,4,4,3,3,3,24,26,26,26,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0
2,3.0,3.0,4.0,4.0,4.0,2.0,1,1,1,1,1,4,4,5,4,4,2,2,2,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0
3,5.0,5.0,4.0,3.0,1.0,2.0,1,1,1,1,1,3,3,3,4,4,6,7,8,8,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0
4,5.0,1.0,2.0,1.0,2.0,1.0,3,3,3,3,3,7,7,7,7,7,3,3,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0
5,5.0,5.0,5.0,5.0,5.0,5.0,2,2,2,2,2,12,12,13,13,13,7,8,8,7,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0
6,1.0,2.0,1.0,1.0,1.0,1.0,3,2,2,1,1,7,6,6,5,6,23,24,23,24,23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1
7,1.0,1.0,1.0,1.0,1.0,1.0,2,1,1,0,1,2,1,1,0,1,4,3,4,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0
8,5.0,5.0,5.0,5.0,4.0,5.0,1,1,1,1,1,3,3,3,3,3,2,2,3,3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1

In [60]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

X_train_scaled.shape

(100397, 106)

In [61]:
view_df = pd.DataFrame(X_train_scaled)
view_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105
0,-1.839312,-1.381729,0.027795,0.283917,-2.039036,-0.309757,-0.796418,-0.835894,-0.867923,-0.862201,-0.847292,-0.321593,-0.349557,-0.363673,-0.364899,-0.356393,-0.175119,-0.195484,-0.209498,-0.213881,-0.208697,-0.077147,-0.099846,-0.062848,-0.14595,-0.0638,-0.0696,-0.081347,-0.083914,-0.056988,-0.10384,-0.062287,-0.078956,-0.065967,-0.065738,-0.085109,-0.075497,-0.127703,-0.273368,-0.077408,-0.212636,-0.080974,-0.103349,-0.146093,-0.062528,-0.12353,-0.051151,-0.077343,-0.062688,-0.256023,-0.172067,-0.075896,-0.183747,-0.065815,-0.065201,-0.081037,-0.111416,-0.271131,-0.074557,-0.05855,-0.149679,-0.089905,-0.091524,-0.178081,-0.561917,-0.057076,-0.083914,-0.079655,4.846963,-0.121731,-0.110911,-0.066119,-0.134771,-0.077927,-0.080349,-0.058034,-0.098101,-0.069093,-0.067548,-0.063642,-0.513128,-0.251062,2.659163,-0.157991,-0.332656,-0.240651,-0.160719,-0.342233,-0.268113,-0.340287,-0.124235,-0.267771,-0.180566,-0.123655,-0.007057,2.143817,-1.257259,-0.513701,-0.636163,-0.547545,1.180579,-0.260369,1.882465,-0.506429,-0.958996,-0.327218
1,1.146188,0.191201,0.868075,0.283917,0.401783,-0.309757,-0.796418,-0.835894,-0.867923,-0.862201,-0.847292,-0.321593,-0.349557,-0.363673,-0.364899,-0.356393,-0.175119,-0.195484,-0.085174,-0.085927,-0.078121,-0.077147,-0.099846,-0.062848,-0.14595,-0.0638,-0.0696,-0.081347,-0.083914,-0.056988,-0.10384,-0.062287,-0.078956,-0.065967,-0.065738,-0.085109,-0.075497,-0.127703,-0.273368,-0.077408,-0.212636,-0.080974,-0.103349,-0.146093,-0.062528,-0.12353,-0.051151,-0.077343,-0.062688,-0.256023,-0.172067,-0.075896,-0.183747,-0.065815,-0.065201,-0.081037,-0.111416,-0.271131,-0.074557,-0.05855,-0.149679,-0.089905,-0.091524,-0.178081,1.779624,-0.057076,-0.083914,-0.079655,-0.206315,-0.121731,-0.110911,-0.066119,-0.134771,-0.077927,-0.080349,-0.058034,-0.098101,-0.069093,-0.067548,-0.063642,1.948832,-0.251062,-0.376058,-0.157991,-0.332656,-0.240651,-0.160719,-0.342233,-0.268113,-0.340287,-0.124235,-0.267771,-0.180566,-0.123655,-0.007057,-0.466458,0.795381,-0.513701,1.571923,-0.547545,-0.847042,-0.260369,-0.531218,1.974611,-0.958996,-0.327218
2,0.399813,0.191201,0.027795,0.283917,0.401783,0.448138,-1.513575,-1.540062,-1.570674,-1.558824,-1.555174,-0.126801,-0.153961,-0.167754,-0.364899,-0.356393,-0.055434,-0.074312,-0.085174,-0.085927,-0.078121,-0.077147,-0.099846,-0.062848,-0.14595,-0.0638,-0.0696,-0.081347,-0.083914,-0.056988,-0.10384,-0.062287,-0.078956,-0.065967,-0.065738,-0.085109,-0.075497,-0.127703,-0.273368,-0.077408,-0.212636,-0.080974,-0.103349,-0.146093,-0.062528,-0.12353,-0.051151,-0.077343,-0.062688,-0.256023,-0.172067,-0.075896,-0.183747,-0.065815,-0.065201,-0.081037,-0.111416,-0.271131,-0.074557,-0.05855,-0.149679,-0.089905,-0.091524,-0.178081,1.779624,-0.057076,-0.083914,-0.079655,-0.206315,-0.121731,-0.110911,-0.066119,-0.134771,-0.077927,-0.080349,-0.058034,-0.098101,-0.069093,-0.067548,-0.063642,-0.513128,3.983082,-0.376058,-0.157991,-0.332656,-0.240651,-0.160719,-0.342233,-0.268113,-0.340287,-0.124235,-0.267771,-0.180566,-0.123655,-0.007057,-0.466458,0.795381,-0.513701,-0.636163,1.826333,-0.847042,-0.260369,-0.531218,-0.506429,1.042757,-0.327218
3,0.399813,-0.595264,-0.812485,0.283917,0.401783,-1.825547,1.355053,1.276609,1.240330,1.227668,0.568473,0.457575,0.628424,0.615922,0.413041,0.604754,-0.534174,-0.559002,-0.582468,-0.597743,-0.600427,-0.077147,10.015427,-0.062848,-0.14595,-0.0638,-0.0696,-0.081347,-0.083914,-0.056988,-0.10384,-0.062287,-0.078956,-0.065967,-0.065738,-0.085109,-0.075497,-0.127703,-0.273368,-0.077408,-0.212636,-0.080974,-0.103349,-0.146093,-0.062528,-0.12353,-0.051151,-0.077343,-0.062688,-0.256023,-0.172067,-0.075896,-0.183747,-0.065815,-0.065201,-0.081037,-0.111416

## NN Attempt - Compile, Train, & Evaluate Default Model

In [64]:
# Define the model
nn_default = tf.keras.models.Sequential()

# Add the masking layer
#nn_default.add(tf.keras.layers.Masking(mask_value=10, input_shape=(X_train_scaled.shape[1],)))

nn_default.add(tf.keras.layers.Dense(units=110, activation='relu', input_dim=X_train_scaled.shape[1]))

nn_default.add(tf.keras.layers.Dense(units=80, activation='relu'))
nn_default.add(tf.keras.layers.Dense(units=80, activation='relu'))
nn_default.add(tf.keras.layers.Dense(units=80, activation='relu'))
nn_default.add(tf.keras.layers.Dense(units=80, activation='relu'))

# Add the output layer
nn_default.add(tf.keras.layers.Dense(units=5, activation='sigmoid'))

# Check the structure of the model
nn_default.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 110)               11770     
                                                                 
 dense_13 (Dense)            (None, 80)                8880      
                                                                 
 dense_14 (Dense)            (None, 80)                6480      
                                                                 
 dense_15 (Dense)            (None, 80)                6480      
                                                                 
 dense_16 (Dense)            (None, 80)                6480      
                                                                 
 dense_17 (Dense)            (None, 5)                 405       
                                                                 
Total params: 40,495
Trainable params: 40,495
Non-trai

In [65]:
# Compile the model
nn_default.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

# Train the model
history = nn_default.fit(X_train_scaled, y_train, epochs=10, verbose=1, batch_size=16)

Epoch 1/10
6275/6275 [==============================] - 20s 3ms/step - loss: 0.0935 - accuracy: 0.6472
Epoch 2/10
6275/6275 [==============================] - 17s 3ms/step - loss: 0.0893 - accuracy: 0.6669
Epoch 3/10
6275/6275 [==============================] - 16s 3ms/step - loss: 0.0882 - accuracy: 0.6721
Epoch 4/10
6275/6275 [==============================] - 22s 3ms/step - loss: 0.0874 - accuracy: 0.6765
Epoch 5/10
6275/6275 [==============================] - 18s 3ms/step - loss: 0.0866 - accuracy: 0.6778
Epoch 6/10
6275/6275 [==============================] - 19s 3ms/step - loss: 0.0859 - accuracy: 0.6826
Epoch 7/10
6275/6275 [==============================] - 18s 3ms/step - loss: 0.0852 - accuracy: 0.6855
Epoch 8/10
6275/6275 [==============================] - 17s 3ms/step - loss: 0.0846 - accuracy: 0.6880
Epoch 9/10
6275/6275 [==============================] - 19s 3ms/step - loss: 0.0840 - accuracy: 0.6904
Epoch 10/10
6275/6275 [==============================] - 17s 3ms/step - l

In [66]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_default.evaluate(X_test_scaled,y_test,verbose=1, batch_size=64)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

523/523 [==============================] - 1s 2ms/step - loss: 0.0887 - accuracy: 0.6726
Loss: 0.0887039452791214, Accuracy: 0.6726229786872864


In [67]:
predictions = pd.DataFrame(nn_default.predict(X_test_scaled))

1046/1046 [==============================] - 2s 1ms/step


In [68]:
predictions = predictions.rename(columns={0: 1, 1: 2, 2: 3, 3: 4, 4: 5})
predictions['y_predict'] = predictions.idxmax(axis=1)
predictions['y_actual'] = (y_test.idxmax(axis=1)).reset_index(drop=True)
predictions[1:10]

,1,2,3,4,5,y_predict,y_actual
1,1.683487e-06,3.011866e-06,0.025309,0.661971,0.268723,4,4
2,7.063877e-04,1.685894e-04,0.079365,0.427940,0.443796,5,5
3,8.092623e-03,3.118327e-02,0.447274,0.365551,0.091978,3,1
4,8.493403e-01,6.123761e-02,0.048171,0.000421,0.012961,1,1
5,1.189109e-06,2.631591e-07,0.010504,0.463526,0.519480,5,4
6,1.108341e-02,4.484427e-02,0.554340,0.308167,0.083180,3,3
7,7.211545e-01,1.346980e-01,0.087025,0.002128,0.021383,1,1
8,4.632223e-07,7.233998e-07,0.016286,0.655357,0.310289,4,5
9,1.152884e-05,5.392627e-05,0.098062,0.727107,0.170967,4,4


In [69]:
difference = 0

for index, row in predictions.iterrows():
    # Get the absolute difference between 'y_predict' and 'y_actual' columns
    diff = abs(row['y_predict'] - row['y_actual'])
    
    # Add the difference to the total
    difference += diff

difference = difference / len(predictions)
print("Mean points delta actual value: ", difference)

Mean points delta actual value:  0.35926014462439493


Hot Ended - Mean points delta actual value:  0.43129239369973577

Half Hot Ended Remove NaN - Mean points delta actual value:  0.36371242455028985

Half Hot Ended Mask NaN - Mean points delta actual value:  0.4334150599593593